<h1 align="center" style="margin-bottom: 20px;">Machine Learning 2024-25</h1>
<h3 align="center" style="margin-top: 20px; margin-bottom: 20px;">To Grant or Not to Grant: Deciding on Compensation Benefits</h3>
<h5 align="center" style="margin-top: 20px; margin-bottom: 0px;">Notebook 4: Open-Ended Section</h5>

### Group 38 - Members:
- Ana Marta Azinheira  - 20240496@novaims.unl.pt
- Braulio Damba - 20240007@novaims.unl.pt
- Catarina Ribeirinha - 20240507@novaims.unl.pt
- Marco Galão  - r20201545@novaims.unl.pt
- Rodrigo Sardinha - 20211627@novaims.unl.pt

# Table of Contents

* [1. Imports](#imports)
  * [1.1. Import Libraries](#import_libraries)
  * [1.2. Import the CV Returns and Best Model](#import_cv_returns_best_model)

* [2. Open-Ended Section](#open_ended)
  * [2.1 Treatment from Notebook 2](#treatment_nb2)
  * [2.2 Treatment and Prediction from Notebook 3](#treatment_prediction_nb3)

# 1. Imports <a id="imports"></a>

## 1.1. Import the Libraries <a class="anchor" id="import_libraries"></a>

In [ ]:
# Path Management
import sys
import os

# Get the current working directory (folder where the notebook is located)
notebook_dir = os.path.dirname(os.path.abspath("__file__"))
# Get the absolute path to the project root (one level up from the notebook's directory)
project_root = os.path.abspath(os.path.join(notebook_dir, '..'))
# Add the project root to sys.path
sys.path.append(project_root)
# Utils
from utils import *

# Visualization Settings
%config InlineBackend.figure_format = 'retina'
sns.set()

# Omit Warnings
import warnings
warnings.filterwarnings("ignore")

In [11]:
import pandas as pd
test = pd.read_csv('predictions_2024-12-20.csv', sep=',', index_col='Claim Identifier')
test.head()

/var/folders/19/rxl10lh147s73036f239kkmm0000gn/T/ipykernel_39373/813817321.py:2: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  test = pd.read_csv('predictions_2024-12-20.csv', sep=',', index_col='Claim Identifier')


,Accident Date,Age at Injury,Alternative Dispute Resolution,Assembly Date,Attorney/Representative,Average Weekly Wage,C-2 Date,C-3 Date,Carrier Name,Carrier Type,...,First Hearing Date Missing,Age Group at Injury,Average Weekly Wage Category,Age * Avg Weekly Wage,Average Weekly Wage by Medical Fee Region,Average Weekly Wage by Gender,Part of Body Category,Nature of Injury Category,Cause of Injury Category,Claim Injury Type
Claim Identifier,,,,,,,,,,,,,,,,,,,,,
6165911,2022-12-24,19.0,0.0,2023-01-02,0,0.0,2023-01-02,NaN,INDEMNITY INSURANCE CO OF,1A. PRIVATE,...,1,Young Adult,Very Low,0.0,172.254176,179.828551,Lower Extremities,Specific Injury,"Fall, Slip or Trip Injury",2. NON-COMP
6166141,2022-11-20,19.0,0.0,2023-01-02,0,0.0,2023-01-02,NaN,A I U INSURANCE COMPANY,1A. PRIVATE,...,1,Young Adult,Very Low,0.0,172.254176,123.526674,Head,Specific Injury,Struck or Injured By,2. NON-COMP
6165907,2022-12-26,59.0,0.0,2023-01-02,0,0.0,2022-12-31,NaN,AMGUARD INSURANCE COMPANY,1A. PRIVATE,...,1,Older Adult,Very Low,0.0,173.103650,123.526674,Trunk,Specific Injury,Striking Against or Stepping On,2. NON-COMP
6166047,2022-12-28,55.0,0.0,2023-01-02,0,0.0,2023-01-02,NaN,INDEMNITY INS. OF N AMERICA,1A. PRIVATE,...,1,Older Adult,Very Low,0.0,172.254176,123.526674,Lower Extremities,Specific Injury,"Fall, Slip or Trip Injury",2. NON-COMP
6166102,2022-12-20,25.0,0.0,2023-01-02,0,0.0,2022-12-31,NaN,NEW HAMPSHIRE INSURANCE CO,1A. PRIVATE,...,1,Young Adult,Very Low,0.0,172.254176,179.828551,Upper Extremities,Specific Injury,Struck or Injured By,2. NON-COMP


## 1.2. Import the CV Returns and Best Model <a class="anchor" id="import_cv_returns_best_model"></a>

In [13]:
# Load the best model (trained)
best_model = joblib.load('best_model.pkl')

# Load the dictionary with the cv returns
cv_returns = joblib.load('cv_returns.pkl')

# Store the cv returns in variables
winsorization_bounds = cv_returns["winsorization_bounds"]
imputers = cv_returns["imputers"]
ordinal_encoders = cv_returns["ordinal_encoders"]
freq_encoders = cv_returns["freq_encoders"]
fill_values_freq_encoding = cv_returns["fill_values_freq_encoding"]
scaler = cv_returns["scaler"]
lencoder = cv_returns["lencoder"]
final_selected_features = cv_returns["final_selected_features"]

# 2. Open-Ended Section <a id="open_ended"></a>

In [14]:
# df_new_inputs = pd.read_csv('df_new_inputs.csv', sep=',', index_col='Claim Identifier')
# df_new_inputs.head()

In [15]:
df_new_inputs = pd.read_csv('../data/test_data.csv', sep=',', index_col='Claim Identifier')
df_new_inputs.head()

,Accident Date,Age at Injury,Alternative Dispute Resolution,Assembly Date,Attorney/Representative,Average Weekly Wage,Birth Year,C-2 Date,C-3 Date,Carrier Name,...,Medical Fee Region,OIICS Nature of Injury Description,WCIO Cause of Injury Code,WCIO Cause of Injury Description,WCIO Nature of Injury Code,WCIO Nature of Injury Description,WCIO Part Of Body Code,WCIO Part Of Body Description,Zip Code,Number of Dependents
Claim Identifier,,,,,,,,,,,,,,,,,,,,,
6165911,2022-12-24,19,N,2023-01-02,N,NaN,2003.0,2023-01-02,NaN,INDEMNITY INSURANCE CO OF,...,IV,NaN,31.0,"FALL, SLIP OR TRIP, NOC",10.0,CONTUSION,54.0,LOWER LEG,10466,1
6166141,2022-11-20,19,N,2023-01-02,N,NaN,2003.0,2023-01-02,NaN,A I U INSURANCE COMPANY,...,IV,NaN,75.0,FALLING OR FLYING OBJECT,10.0,CONTUSION,10.0,MULTIPLE HEAD INJURY,11691,1
6165907,2022-12-26,59,N,2023-01-02,N,0.0,1963.0,2022-12-31,NaN,AMGUARD INSURANCE COMPANY,...,III,NaN,68.0,STATIONARY OBJECT,49.0,SPRAIN OR TEAR,62.0,BUTTOCKS,10604,0
6166047,2022-12-28,55,N,2023-01-02,N,0.0,0.0,2023-01-02,NaN,INDEMNITY INS. OF N AMERICA,...,IV,NaN,25.0,FROM DIFFERENT LEVEL (ELEVATION),10.0,CONTUSION,53.0,KNEE,11411,6
6166102,2022-12-20,25,N,2023-01-02,N,0.0,1997.0,2022-12-31,NaN,NEW HAMPSHIRE INSURANCE CO,...,IV,NaN,79.0,OBJECT BEING LIFTED OR HANDLED,40.0,LACERATION,37.0,THUMB,11212,5


## 2.1 Treatment from Notebook 2 <a id="treatment_nb2"></a>

In [16]:
# -------------------- Drop columns with a lot of missing values and replace placeholder values with NaN --------------------

# Drop columns that are not needed
df_new_inputs.drop(columns=["OIICS Nature of Injury Description", "IME-4 Count"], inplace=True)

# Replace placeholder values with NaN
placeholder_replacements = {
    "Carrier Type": "UNKNOWN",
    "Gender": "U",
    "Medical Fee Region": "UK",
    "Alternative Dispute Resolution": "U",
    "County of Injury": "UNKNOWN"
}
for col, placeholder in placeholder_replacements.items():
    df_new_inputs[col].replace(placeholder, np.nan, inplace=True)

# -------------------- Adjust data types for columns --------------------

# Define column groups for type adjustments
columns_object_to_datetime = ['Accident Date', 'Assembly Date', 'C-2 Date', 'C-3 Date', 'First Hearing Date']
code_columns = ['Industry Code', 'WCIO Cause of Injury Code', 'WCIO Nature of Injury Code', 'WCIO Part Of Body Code']
columns_float_to_int = ['Age at Injury', 'Birth Year', 'Number of Dependents']
columns_object_to_binary = ['Alternative Dispute Resolution', 'Attorney/Representative', 'COVID-19 Indicator']

# Convert object columns to datetime (missing dates become NaT)
for col in columns_object_to_datetime:
    df_new_inputs[col] = pd.to_datetime(df_new_inputs[col])

# Convert float columns with codes to object type (via Int64 for nullable integers)
for col in code_columns:
    df_new_inputs[col] = df_new_inputs[col].astype('Int64').astype(object)

# Convert float columns to integers (using Int64 for null handling)
for col in columns_float_to_int:
    df_new_inputs[col] = df_new_inputs[col].astype('Int64')

# Convert binary object columns to binary integers (Y/N to 1/0)
for col in columns_object_to_binary:
    df_new_inputs[col] = df_new_inputs[col].map({'Y': 1, 'N': 0}).astype('Int64')

# Handle gender conversion: convert 'Gender' column to binary and drop original
df_new_inputs["Male"] = df_new_inputs["Gender"].map({'M': 1, 'F': 0}).astype('Int64')
df_new_inputs.drop(columns=['Gender'], inplace=True)

# -------------------- Adjust date columns for consistency --------------------

# Apply the date adjustment function
df_new_inputs = df_new_inputs.apply(adjust_dates, axis=1)
df_new_inputs["Male"] = df_new_inputs["Male"].astype('Int64') # Convert 'Male' column back to nullable integers

# -------------------- Additional adjustments for specific columns --------------------

# Convert specific object columns to nullable integers
columns_object_to_int = ["Birth Year", "Alternative Dispute Resolution"]
for col in columns_object_to_int:
    df_new_inputs[col] = df_new_inputs[col].astype('Int64')

# Drop columns that are no longer needed
df_new_inputs.drop(columns=['Birth Year', 'Number of Dependents'], inplace=True)

# -------------------- Validate and clean ZIP codes --------------------

# Define the valid zip code pattern (5 digits)
valid_zip_pattern = r'^\d{5}$'
df_new_inputs['Zip Code'] = df_new_inputs['Zip Code'].apply(lambda x: x if re.match(valid_zip_pattern, str(x)) else np.nan)

# -------------------- Handle "WCIO Part Of Body Code" adjustments --------------------

# Convert "WCIO Part Of Body Code" column to numeric for processing
df_new_inputs["WCIO Part Of Body Code"].apply(pd.to_numeric)

# Adjust codes with invalid values
condition = df_new_inputs['WCIO Part Of Body Code'].isin([-9, 90])
df_new_inputs.loc[df_new_inputs['WCIO Part Of Body Code'] == -9, 'WCIO Part Of Body Code'] = 90
df_new_inputs.loc[condition, 'WCIO Part Of Body Description'] = 'Multiple Body Parts'

# Convert "WCIO Part Of Body Code" back to object
df_new_inputs['WCIO Part Of Body Code'] = df_new_inputs['WCIO Part Of Body Code'].astype('object')

## 2.2 Treatment and Prediction from Notebook 3 <a id="treatment_prediction_nb3"></a>

In [17]:
# -------------------- Treatment --------------------

# Winsorization
for col in feats_dict["winsorization"]:
    # Apply the same bounds to the new inputs
    df_new_inputs, _ = winsorization(df_new_inputs, col, bounds=winsorization_bounds[col])

# Missing values imputation
df_new_inputs, _ = impute_missing_values(df_new_inputs, feats_dict, imputers=imputers)

# Feature engineering
df_new_inputs = create_features(df_new_inputs)

# Save this dataset to be able to create analysis on all trated variables and not encoded and not standardized
df_new_inputs_with_predictions = df_new_inputs.copy()

# Drop description and date columns
df_new_inputs.drop(columns = feats_dict["codes_drop"] + feats_dict["descriptions_drop"] + feats_dict["dates_drop"], inplace=True)

# Ordinal encoding
_, df_new_inputs, _ = ordinal_encoder(feats_dict, data=df_new_inputs, encoders=ordinal_encoders)

# Frequency encoding
_, df_new_inputs, _, _ = frequency_encoder(feats_dict, data=df_new_inputs, encoders=freq_encoders, fill_values=fill_values_freq_encoding)

# Data scaling
_, df_new_inputs, _ = scale_data(data=df_new_inputs, scaler=scaler)

# Feature selection
df_new_inputs = df_new_inputs[final_selected_features]

# -------------------- Prediction --------------------

# Predict the 'Claim Injury Type' for the new inputs dataset by using the trained model
predictions = best_model.predict(df_new_inputs)

# Decode the predicted labels back to their original categorical values
decoded_predictions = lencoder.inverse_transform(predictions)

# Add the predictions as a new column to the original DataFrame
df_new_inputs_with_predictions['Claim Injury Type'] = decoded_predictions

# Display the DataFrame
df_new_inputs_with_predictions.head()

,Accident Date,Age at Injury,Alternative Dispute Resolution,Assembly Date,Attorney/Representative,Average Weekly Wage,C-2 Date,C-3 Date,Carrier Name,Carrier Type,...,First Hearing Date Missing,Age Group at Injury,Average Weekly Wage Category,Age * Avg Weekly Wage,Average Weekly Wage by Medical Fee Region,Average Weekly Wage by Gender,Part of Body Category,Nature of Injury Category,Cause of Injury Category,Claim Injury Type
Claim Identifier,,,,,,,,,,,,,,,,,,,,,
6165911,2022-12-24,19.0,0.0,2023-01-02,0,0.0,2023-01-02,NaT,INDEMNITY INSURANCE CO OF,1A. PRIVATE,...,1,Young Adult,Very Low,0.0,172.043979,179.619562,Lower Extremities,Specific Injury,"Fall, Slip or Trip Injury",2. NON-COMP
6166141,2022-11-20,19.0,0.0,2023-01-02,0,0.0,2023-01-02,NaT,A I U INSURANCE COMPANY,1A. PRIVATE,...,1,Young Adult,Very Low,0.0,172.043979,123.410058,Head,Specific Injury,Struck or Injured By,2. NON-COMP
6165907,2022-12-26,59.0,0.0,2023-01-02,0,0.0,2022-12-31,NaT,AMGUARD INSURANCE COMPANY,1A. PRIVATE,...,1,Older Adult,Very Low,0.0,172.831337,123.410058,Trunk,Specific Injury,Striking Against or Stepping On,2. NON-COMP
6166047,2022-12-28,55.0,0.0,2023-01-02,0,0.0,2023-01-02,NaT,INDEMNITY INS. OF N AMERICA,1A. PRIVATE,...,1,Older Adult,Very Low,0.0,172.043979,123.410058,Lower Extremities,Specific Injury,"Fall, Slip or Trip Injury",2. NON-COMP
6166102,2022-12-20,25.0,0.0,2023-01-02,0,0.0,2022-12-31,NaT,NEW HAMPSHIRE INSURANCE CO,1A. PRIVATE,...,1,Young Adult,Very Low,0.0,172.043979,179.619562,Upper Extremities,Specific Injury,Struck or Injured By,2. NON-COMP


In [19]:
# # Export predictions from new inputs
# df_new_inputs.to_csv('df_new_inputs_predictions.csv')

In [4]:
test = joblib.load('../results/objects_final_predictions.pkl')

In [5]:
test["final_selected_features"]

['Age at Injury',
 'Attorney/Representative',
 'Average Weekly Wage',
 'Industry Code',
 'Zip Code',
 'C-2 Date Missing',
 'First Hearing Date Missing',
 'Age * Avg Weekly Wage',
 'Average Weekly Wage by Medical Fee Region',
 'Part of Body Category',
 'Cause of Injury Category']

# Backend (API and File Upload)

In [ ]:
from flask import Flask, request, jsonify, send_file, render_template
import pandas as pd
import joblib
import os
from io import BytesIO

# Load trained model and encoders
best_model = joblib.load('best_model.pkl')
lencoder = joblib.load('label_encoder.pkl')
feats_dict = joblib.load('feats_dict.pkl')

app = Flask(__name__)

UPLOAD_FOLDER = './uploads'
RESULTS_FOLDER = './results'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
os.makedirs(RESULTS_FOLDER, exist_ok=True)

@app.route('/')
def home():
    return render_template('upload.html')  # Create a simple HTML upload page

@app.route('/upload', methods=['POST'])
def upload_and_predict():
    if 'file' not in request.files:
        return jsonify({"error": "No file uploaded"}), 400

    file = request.files['file']
    if file.filename == '':
        return jsonify({"error": "No selected file"}), 400

    # Save uploaded file
    filepath = os.path.join(UPLOAD_FOLDER, file.filename)
    file.save(filepath)

    # Load and preprocess data
    try:
        df_new_inputs = pd.read_csv(filepath, sep=',', index_col='Claim Identifier')
        df_predictions = process_and_predict(df_new_inputs)
    except Exception as e:
        return jsonify({"error": str(e)}), 500

    # Save predictions to file
    result_filepath = os.path.join(RESULTS_FOLDER, 'predictions.csv')
    df_predictions.to_csv(result_filepath)

    return send_file(result_filepath, as_attachment=True, download_name='predictions.csv')

def process_and_predict(df_new_inputs):
    # Preprocess: replicate your existing code here
    # (Include all preprocessing, encoding, and feature engineering steps)
    
    # Predict
    predictions = best_model.predict(df_new_inputs)
    decoded_predictions = lencoder.inverse_transform(predictions)
    df_new_inputs['Claim Injury Type'] = decoded_predictions
    return df_new_inputs

if __name__ == '__main__':
    app.run(debug=True)


# Dashboard (Visualization with Dash)

from dash import Dash, dcc, html
import plotly.express as px
import pandas as pd

app = Dash(__name__)

# Load predictions for visualization
df_predictions = pd.read_csv('./results/predictions.csv')

# Layout
app.layout = html.Div([
    html.H1("Prediction Results Dashboard"),
    dcc.Graph(id='class-distribution'),
    dcc.Graph(id='summary-statistics')
])

@app.callback(
    [dcc.Graph(id='class-distribution'),
     dcc.Graph(id='summary-statistics')]
)
def update_dashboard():
    # Class Distribution
    class_dist_fig = px.bar(
        df_predictions['Claim Injury Type'].value_counts(),
        title="Class Distribution of Predictions"
    )

    # Summary Statistics
    summary_stats_fig = px.box(
        df_predictions,
        x='Claim Injury Type',
        y='Some_Numeric_Feature',  # Replace with actual numeric column
        title="Feature Distribution by Class"
    )

    return class_dist_fig, summary_stats_fig

if __name__ == '__main__':
    app.run_server(debug=True)

In [ ]:
# # Install dependencies
# pip install flask dash pandas joblib plotly

# # Start the Flask backend
# pip install flask dash pandas joblib plotly

# # Start the Dash dashboard
# pip install flask dash pandas joblib plotly